# Preparação do Cadastro inicial de Pacientes de Avaí (SP) para a JCAFB-2025
Outubro de 2024

## Dados fornecidos pela Prefeitura de Avaí

Os dados foram fornecidos em arquivos (formato PDF) contendo os dados dos Pacientes de Crianças e Idosos, distribuídos em 7 Micro Áreas:

- Microarea 1 - Crianças.pdf
- Microarea 1 - Idosos.pdf
- Microarea 2 - Crianças.pdf
- Microarea 2 - Idosos.pdf
- Microarea 3 - Crianças.pdf
- Microarea 3 - Idosos.pdf
- Microarea 5 - Crianças.pdf
- Microarea 5 - Idosos.pdf
- Microarea 6 - Crianças.pdf
- Microarea 6 - Idosos.pdf
- Microarea 7 - Crianças.pdf
- Microarea 7 - Idosos.pdf


## Conversão do conteúdo dos aquivos fornecidos (.pdf) para Planilhas do Microsoft Excel (.xlsx)

Utilizando o método de copia e cola, os conteúdos de cada arquivo apresentado foram transferidos para Planilhas do Microsoft Excel, gerando um arquivo um arquivo .xlsx para cada arquivo .pdf apresentado:

- Microarea 1 - Criancas.xlsx
- Microarea 1 - Idosos.xlsx
- Microarea 2 - Criancas.xlsx
- Microarea 2 - Idosos.xlsx
- Microarea 3 - Criancas.xlsx
- Microarea 3 - Idosos.xlsx
- Microarea 5 - Criancas.xlsx
- Microarea 5 - Idosos.xlsx
- Microarea 6 - Criancas.xlsx
- Microarea 6 - Idosos.xlsx
- Microarea 7 - Criancas.xlsx
- Microarea 7 - Idosos.xlsx


No processo de colagem do conteúdo, uma única coluna foi utilizada ("**Texto PDF**) onde várias linhas (número variável) foram ocupadas.

## Processamento dos conteúdos dos arquivos .xlsx

In [1]:
import pandas as pd

### Funções de Suporte (1)

In [2]:
def string_end_clean(string):

    if string[-1] == ' ':
        string = string[:-1]
    if string[-1] == '.':
        string = string[:-1]
    if string[-1] == '-':
        string = string[:-1]
    if string[-1] == ' ':
        string = string[:-1]

    return string


In [3]:
def accented_character_clean(string):

    string = string.replace("á", "a")
    string = string.replace("Á", "A")
    string = string.replace("ã", "a")
    string = string.replace("â", "a")
    string = string.replace("à", "a")
    string = string.replace("é", "e")
    string = string.replace("í", "i")
    string = string.replace("Í", "I")
    string = string.replace("ó", "o")
    string = string.replace("õ", "o")
    string = string.replace("ô", "o")
    string = string.replace("ú", "u")
    string = string.replace("ç", "c")

    return string


In [4]:
def string_reformat(string):

    string = string.title()

    string = string.replace(" De ", " de ")
    string = string.replace(" Da ", " da ")
    string = string.replace(" Do ", " do ")
    string = string.replace(" Dos ", " dos ")

    return string


In [5]:
def cadastro_file_process(xls_in_path, sheet):

    xls_in = pd.ExcelFile(xls_in_path)
    table = pd.read_excel(xls_in, sheet_name = sheet)

    # Novas colunas inseridas no data frame
    # table.insert(0, 'Texto PDF', None)
    # Coluna contendo os dados originais provenientes do arquivo .pdf.
    table.insert(1, 'Flag', None)
    table.insert(2, 'Nome', None)
    table.insert(3, 'ID', None)
    table.insert(4, 'Nascimento', None)
    table.insert(5, 'Sexo', None)
    table.insert(6, 'Logradouro', None)
    table.insert(7, 'Numero', None)
    table.insert(8, 'Complemento', None)
    table.insert(9, 'Bairro', None)
    table.insert(10, 'Telefone', None)

    # Marcação da última linha das informações de cada Paciente.
    # Flag == '--'
    table.loc[(table['Texto PDF']=='CDS') |
              (table['Texto PDF']=='PEC'), 'Flag'] = '--'

    # Marcação da primeira linha das informações de cada Paciente.
    # Flag == '**'
    # Esta marcação indica a linha onde o nome do Paciente está armazenado.
    for i, row in table.iterrows():
        if i == 0:
            table['Flag'].values[i] = '**'
        if row['Flag'] == '--':
            try:
                table['Flag'].values[i + 1] = '**'
            except:
                pass

    # Marcação a linha onde está armazenado o número do CPF do Paciente.
    # Flag == 'cpf'
    # Caso esta não seja a segunda linha das informações do Paciente, a segunda linha é marcada com o
    # Flag == '*c' indicando que a segunda linha contêm a continuação do nome do Paciente.
    for i, row in table.iterrows():
        if row['Flag'] == '**':
            if table['Texto PDF'].values[i + 1][0:4] == 'CPF:':
                table['Flag'].values[i + 1] = 'cpf'
            else:
                if table['Texto PDF'].values[i + 2][0:4] == 'CPF:':
                    table['Flag'].values[i + 2] = 'cpf'
                    table['Flag'].values[i + 1] = '*c'

    # Marcação a linha onde está armazenado o número do CNS (Cartão Nacional de Saude) do Paciente.
    # Flag == 'cns'
    # Caso esta não seja a segunda linha das informações do Paciente, a segunda linha é marcada com o
    # Flag == '*c' indicando que a segunda linha contêm a continuação do nome do Paciente.
    for i, row in table.iterrows():
        if row['Flag'] == '**':
            if table['Texto PDF'].values[i + 1][0:4] == 'CNS:':
                table['Flag'].values[i + 1] = 'cns'
            else:
                if table['Texto PDF'].values[i + 2][0:4] == 'CNS:':
                    table['Flag'].values[i + 2] = 'cns'
                    table['Flag'].values[i + 1] = '*c'

    # Obtenção do Nome do Paciente
    for i, row in table.iterrows():
        if row['Flag'] == '**':
            if table['Flag'].values[i + 1] == '*c':
                name = row['Texto PDF'] + ' ' + table['Texto PDF'].values[i + 1]
            else:
                name = row['Texto PDF']
            name = string_reformat(name)
            name = accented_character_clean(name)
            table['Nome'].values[i] = name

    # Obtenção do número do CPF do Paciente
    for i, row in table.iterrows():
        if row['Flag'] == 'cpf':
            id = row['Texto PDF'][5:19]
            table['ID'].values[i] = id

    # Obtenção do número do CNS do Paciente
    for i, row in table.iterrows():
        if row['Flag'] == 'cns':
            id = row['Texto PDF'][5:20]
            table['ID'].values[i] = id

    # Obtenção do Gênero (Sexo) e da Data de Nascimento do Paciente
    for i, row in table.iterrows():
        text = str(row['Texto PDF'])
        sex = False
        if text.find("Feminino") != -1:
            sex = 'Feminino'
        if text.find("Masculino") != -1:
            sex = 'Masculino'
        if sex:
            table['Sexo'].values[i] = sex
            index = text.index('/')
            if index > 0:
                 table['Nascimento'].values[i] = text[index - 2:index + 8]

    # Obtenção do Número do Telefone do Paciente
    for i, row in table.iterrows():
        tel = False
        text = str(row['Texto PDF'])
        index = text.find("(1")
        if index != -1:
            tel = text[index:index + 15]
        if tel:
            table['Telefone'].values[i] = tel

    # Obtenção das informações completas do Endereço do Paciente

    for i, row in table.iterrows():
        logradouro = False
        numero = False
        complemento = False
        bairro = False
        text = str(row['Texto PDF'])

        # Obtenção do texto contendo todas as possíveis informações do Endereço do Paciente

        index = text.find("Rua") 
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        index = text.find("Avenida")
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        index = text.find("Travessa")
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        index = text.find("Praça")
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        index = text.find("Viela")
        if index != -1:
            logradouro = text[index:]
            if table['Flag'].values[i + 1] != '--':
                logradouro = logradouro + ' ' + table['Texto PDF'].values[i + 1]

        # Segmentação das informações de Endereço do Paciente

        if logradouro:

            # Exclusão das informações da Cidade (Avaí) e outras informações, caso tenham sido coletadas
            index = logradouro.find(', Avaí')
            if index != -1:
                logradouro = logradouro[:index]
            if logradouro[-1] == ',':
                logradouro = logradouro[:-1]

            # Identificação de Endereços localizados no Bairro "Centro".

            index = logradouro.find('CENTRO')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Centro'
            index = logradouro.find('Centro')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Centro'

            # Identificação de Endereços localizados no Bairro "Vila Oliveira".

            index = logradouro.find('VILA OLIVEIRA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Vila Oliveira'
            index = logradouro.find('Vila Oliveira')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Vila Oliveira'

            # Identificação de Endereços localizados no Bairro "Núcleo Habitacional Eunice Andrade Moreira".
            index = logradouro.find('NÚCLEO HABITACIONAL EUNICE ANDRADE')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Eunice Andrade Moreira'
            index = logradouro.find('NUCLEO HABITACIONAL EUNICE ANDRADE')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Eunice Andrade Moreira'
            index = logradouro.find('NÚCLEO HABITACIONAL EUNICE')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Eunice Andrade Moreira'
            index = logradouro.find('Núcleo Habitacional Eunice Andrade Moreira')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Eunice Andrade Moreira'

            # Identificação de Endereços localizados no Bairro "Núcleo Habitacional Julio Rocha".

            index = logradouro.find('NÚCLEO HABITACIONAL JULIO ROCHA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'
            index = logradouro.find('NUCLEO HABITACIONAL JULIO ROCHA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'
            index = logradouro.find('Núcleo Habitacional Julio Rocha')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Nucleo Hab Julio Rocha'
            index = logradouro.find('Núcleo Habitacional Julio Rocha')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'
            index = logradouro.find('JULIO ROCHA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'
            index = logradouro.find('Nucleo Hab Julio Rocha')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Núcleo Habitacional Julio Rocha'

            # Identificação de Endereços localizados no Bairro "Cohab".

            index = logradouro.find('COHAB')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Cohab'
            index = logradouro.find('COAHB')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Cohab'
            index = logradouro.find('Cohab')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Cohab'
            index = logradouro.find('Coab')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Cohab'

            # Identificação de Endereços localizados no Bairro "Conjunto Habitacional Manoel Rodrigues".

            index = logradouro.find('CONJUNTO HABITACIONAL MANOEL RODRIGUES')

            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Conjunto Habitacional Manoel Rodrigues'
            index = logradouro.find('Conjunto Habitacional Manoel Rodrigues')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Conjunto Habitacional Manoel Rodrigues'

            # Identificação de Endereços localizados no Bairro "Baixada".

            index = logradouro.find('BAIXADA')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Baixada'
            index = logradouro.find('Baixada')
            if index != -1:
                logradouro = logradouro[:index]
                bairro = 'Baixada'

            # Eliminação de caracteres espúrios do final do texto do Logradouro
            logradouro = string_end_clean(logradouro)

            # Obtenção das informações do Complemento da Residência dos Pacientes

            index = logradouro.find('CASA')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('Casa')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('FUNDOS')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('FUMDOS')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('Fundos')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('FRENTE')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('Frente')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('SOBRADO')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]
            index = logradouro.find('Sobrado')
            if index != -1:
                complemento = logradouro[index:]
                logradouro = logradouro[:index]

            # Eliminação de caracteres espúrios do final do texto do Logradouro
            logradouro = string_end_clean(logradouro)

            # Obtenção das informações do Número da Residência dos Pacientes

            index = logradouro.find(', ')
            if index != -1:
                numero = logradouro[index + 2:]
                logradouro = logradouro[:index]

        # Armazenamento das informações do Endereço do Paciente

        if logradouro:
            logradouro = string_reformat(logradouro)
            table['Logradouro'].values[i] = logradouro
        if numero:
            numero = numero.replace('S/N', 'sn')
            table['Numero'].values[i] = numero
        if complemento:
            complemento = string_reformat(complemento)
            table['Complemento'].values[i] = complemento
        if bairro:
            table['Bairro'].values[i] = bairro

    return table

### Processamento dos arquivos de Cadastro

In [6]:
xls_out_path = 'data/avai_cadastro.xlsx'

In [7]:
xls_in_path = 'data/Microarea 1 - Criancas.xlsx'
sheet = 'MC1_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
table.to_excel(xls_out_path, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 1 - Idosos.xlsx'
sheet = 'MC1_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 2 - Criancas.xlsx'
sheet = 'MC2_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 2 - Idosos.xlsx'
sheet = 'MC2_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 3 - Criancas.xlsx'
sheet = 'MC3_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 3 - Idosos.xlsx'
sheet = 'MC3_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 5 - Criancas.xlsx'
sheet = 'MC5_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 5 - Idosos.xlsx'
sheet = 'MC5_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 6 - Criancas.xlsx'
sheet = 'MC6_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 6 - Idosos.xlsx'
sheet = 'MC6_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 7 - Criancas.xlsx'
sheet = 'MC7_Criancas'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)

xls_in_path = 'data/Microarea 7 - Idosos.xlsx'
sheet = 'MC7_Idosos'
table = cadastro_file_process(xls_in_path, sheet)
with pd.ExcelWriter(xls_out_path, mode='a') as writer:
    table.to_excel(writer, sheet_name=sheet, index=False)


## Criação do Cadastro inicial de Pacientes da JCAFB-2025

### Funções de Suporte (2)

In [8]:
def jcafb_cadastro_file_process(xls_in_path, sheet, categoria, casdastro):

    xls_in = pd.ExcelFile(xls_in_path)
    table = pd.read_excel(xls_in, sheet_name = sheet, na_filter=False)

    nome = ''
    id_ = ''
    nascimento = ''
    sexo = ''
    logradouro = 'None'
    numero = ''
    complemento = ''
    bairro = 'None'
    telefone = ''
    fonte = sheet
    street = ''
    contact = ''
    for i, row in table.iterrows():
        if row['Flag'] == '**':
            nome = row['Nome']
        elif row['Flag'] == '--':
            new_row = {'Nome': nome,
                       'ID': id_,
                       'Nascimento': nascimento,
                       'Sexo': sexo,
                       'Logradouro': logradouro,
                       'Número': numero,
                       'Complemento': complemento,
                       'Bairro': bairro,
                       'Telefone': telefone,
                       'Categoria': categoria,
                       'Fonte': fonte,
                       'Street': street,
                       'Contact': contact
                      }
            cadastro.loc[len(cadastro)] = new_row
            nome = ''
            id_ = ''
            nascimento = ''
            sexo = ''
            logradouro = 'None'
            numero = ''
            complemento = ''
            bairro = 'None'
            telefone = ''
            fonte = sheet
            street = ''
            contact = ''
        else:
            if row['ID'] != '':
                id_ = row['ID']
            if row['Nascimento'] != '':
                nascimento = row['Nascimento']
            if row['Sexo'] != '':
                sexo = row['Sexo']
            if row['Logradouro'] != '':
                logradouro = row['Logradouro']
            if row['Numero'] != '':
                numero = row['Numero']
            if row['Complemento'] != '':
                complemento = row['Complemento']
            if row['Bairro'] != '':
                bairro = row['Bairro']
            if row['Telefone'] != '':
                telefone = row['Telefone']

    return cadastro

### Criação do arquivo do Cadastro inicial de Pacientes da JCAFB-2025

In [9]:
xls_in_path = 'data/avai_cadastro.xlsx'

jcafb_cadastro_path = 'data/jcafb_cadastro.xlsx'
jcafb_cadastro_sheet='Cadastro JCAFB-2025'

column_names=['Nome', 'ID', 'Nascimento', 'Sexo', 'Logradouro', 'Número', 'Complemento',
              'Bairro', 'Telefone', 'Categoria', 'Fonte']
cadastro = pd.DataFrame(columns=column_names)

sheet = 'MC1_Criancas'
categoria = 'Criança'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC1_Idosos'
categoria = 'Idoso'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC2_Criancas'
categoria = 'Criança'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC2_Idosos'
categoria = 'Idoso'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC3_Criancas'
categoria = 'Criança'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC3_Idosos'
categoria = 'Idoso'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC5_Criancas'
categoria = 'Criança'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC5_Idosos'
categoria = 'Idoso'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC6_Criancas'
categoria = 'Criança'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC6_Idosos'
categoria = 'Idoso'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC7_Criancas'
categoria = 'Criança'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

sheet = 'MC7_Idosos'
categoria = 'Idoso'
cadastro = jcafb_cadastro_file_process(xls_in_path, sheet, categoria, cadastro)

ref_logradouros = pd.read_excel('data/jcafb_ref_logradouros.xlsx',
                                sheet_name = 'Ref Logradouros JCAFB-2025',
                                na_filter=False)
for i, row in cadastro.iterrows():
    ref_index = ref_logradouros.index[ref_logradouros['from_logradouro'] == row['Logradouro']].tolist()
    cadastro['Logradouro'].values[i] = ref_logradouros['to_logradouro'].values[ref_index][0]

ref_bairros = pd.read_excel('data/jcafb_ref_bairros.xlsx',
                            sheet_name = 'Ref Bairros JCAFB-2025',
                            na_filter=False)
for i, row in cadastro.iterrows():
    ref_index = ref_bairros.index[ref_bairros['from_bairro'] == row['Bairro']].tolist()
    cadastro['Bairro'].values[i] = ref_bairros['to_bairro'].values[ref_index][0]

cadastro.insert(11, 'Street', None)
for i, row in cadastro.iterrows():
    if row['Logradouro'] != '':
        cadastro['Street'].values[i] = row['Logradouro'] + ' (' + row['Bairro'] + ')'

cadastro.insert(12, 'Contact', None)
for i, row in cadastro.iterrows():
    if row['Logradouro'] != '':
        cadastro['Contact'].values[i] = row['Logradouro'] + ', ' + row['Número'] + '/' + row['Complemento'] + ' (' + row['Bairro'] + ')'

cadastro.to_excel(jcafb_cadastro_path, sheet_name=jcafb_cadastro_sheet, index=True)


## Análise de Dados

### Identificação das Fontes dos Dados Apresentados

In [10]:
import pandas as pd

jcafb_cadastro_path = 'data/jcafb_cadastro.xlsx'
sheet = 'Cadastro JCAFB-2025'

xls_in = pd.ExcelFile(jcafb_cadastro_path)
jcafb_cadastro = pd.read_excel(xls_in, sheet_name = sheet, index_col=0, na_filter=False)


In [11]:
jcafb_cadastro.groupby('Fonte')['Nome'].count()

Fonte
MC1_Criancas     38
MC1_Idosos      114
MC2_Criancas     26
MC2_Idosos       75
MC3_Criancas     53
MC3_Idosos      138
MC5_Criancas     43
MC5_Idosos      104
MC6_Criancas     28
MC6_Idosos       83
MC7_Criancas     36
MC7_Idosos      105
Name: Nome, dtype: int64

### Identificação das Categorias dos Pacientes

In [12]:
jcafb_cadastro.groupby('Categoria')['Nome'].count()

Categoria
Criança    224
Idoso      619
Name: Nome, dtype: int64

### Identificação dos Bairros da Cidade

In [13]:
bairros = jcafb_cadastro.groupby('Bairro')['Nome'].count()

bairros.to_excel('data/jcafb_bairros.xlsx', sheet_name='Bairros JCAFB-2025', index=True)

bairros.head(50)

Bairro
                         2
Baixada                164
CH Manoel Rodrigues      1
Centro                 335
Cohab                    9
NH Eunice Andrade      129
NH Julio Rocha          38
Vila Oliveira          165
Name: Nome, dtype: int64

### Idenfificação dos Logradouros da Cidade

In [14]:
jcafb_logradouros_path = 'data/jcafb_logradouros.xlsx'
jcafb_logradouros_sheet='Logradouros JCAFB-2025'

logradouros = jcafb_cadastro.groupby('Logradouro')['Logradouro'].count()

logradouros.to_excel(jcafb_logradouros_path, sheet_name=jcafb_logradouros_sheet, index=True)

logradouros

Logradouro
                                        2
Avenida Camilo Mady                    10
Avenida Doutor Alberto Salles          57
Avenida Manoel Rodrigues               13
Praca Major Gasparino de Quadros        1
Rua Amaro Militao de Lima              20
Rua Antonio Bernardo da Cunha          59
Rua Antonio Domingos de Oliveira       28
Rua Antonio Manoel da Silva            15
Rua Antonio Venancio                   29
Rua Barao do Rio Branco                91
Rua Benedito Antonio de Camargo        26
Rua Cidonio Esgalha                     8
Rua Coronel Juvencio Silva             57
Rua Coronel Tosoni de Carlis           30
Rua Domingos Zulian                    33
Rua Doutor Osvaldo Cruz                48
Rua Doutor Pedro da Rocha Braga        18
Rua Doutor Pedro de Toledo              5
Rua Duque de Caxias                    31
Rua Eduardo Mendes                      4
Rua Emilio Tieppo                       5
Rua Fermino dos Santos                  8
Rua Gomes Berriel Filho

### Idenfificação dos Logradouros (Bairros) da Cidade

In [15]:
jcafb_streets_path = 'data/jcafb_streets.xlsx'
jcafb_streets_sheet='Streets JCAFB-2025'

streets = jcafb_cadastro.groupby('Street')['Nome'].count()

streets.to_excel(jcafb_streets_path, sheet_name=jcafb_streets_sheet, index=True)

streets

Street
                                                        2
Avenida Camilo Mady (Centro)                           10
Avenida Doutor Alberto Salles (Baixada)                23
Avenida Doutor Alberto Salles (CH Manoel Rodrigues)     1
Avenida Doutor Alberto Salles (Centro)                 25
                                                       ..
Travessa 15 de Novembro (Vila Oliveira)                 1
Travessa Barao do Rio Branco (Baixada)                  3
Travessa Leonilda Coelho de Azevedo (Baixada)           7
Travessa Manoel Ramos (Vila Oliveira)                   8
Viela Manoel Mendes (Baixada)                           3
Name: Nome, Length: 68, dtype: int64

### Identificação das Residências dos Pacientes

In [16]:
jcafb_contacts_path = 'data/jcafb_contacts.xlsx'
jcafb_contacts_sheet='Contacts JCAFB-2025'

contacts = jcafb_cadastro.groupby('Contact')['Nome'].count()

contacts.to_excel(jcafb_contacts_path, sheet_name=jcafb_contacts_sheet, index=True)

contacts

Contact
                                                        2
Avenida Camilo Mady, 102/Casa (Centro)                  1
Avenida Camilo Mady, 116/Casa (Centro)                  3
Avenida Camilo Mady, 15/Casa (Centro)                   1
Avenida Camilo Mady, 34/Casa (Centro)                   2
                                                       ..
Travessa Manoel Ramos, 50. TRAVESSA/ (Vila Oliveira)    1
Travessa Manoel Ramos, 68/ (Vila Oliveira)              2
Viela Manoel Mendes, 31/Casa (Baixada)                  1
Viela Manoel Mendes, 40/Casa (Baixada)                  1
Viela Manoel Mendes, 838/Casa (Baixada)                 1
Name: Nome, Length: 647, dtype: int64

### Identificação dos Pacientes Homônimos

In [17]:
import pandas as pd

jcafb_cadastro_path = 'data/jcafb_cadastro.xlsx'
jcafb_cadastro_sheet = 'Cadastro JCAFB-2025'

xls_in = pd.ExcelFile(jcafb_cadastro_path)
jcafb_cadastro = pd.read_excel(xls_in, sheet_name=jcafb_cadastro_sheet, index_col=0, na_filter=False)

jcafb_homonimos_path = 'data/jcafb_homonimos.xlsx'
jcafb_homonimos_sheet='Homônimos JCAFB-2025'

column_names=['Nome', 'ID', 'Nascimento', 'Fonte']
homonimos = pd.DataFrame(columns=column_names)

names = jcafb_cadastro.groupby(['Nome'])['ID'].aggregate('count').reset_index()

for i, row in names.iterrows():

    if row['ID'] > 1:

        print(row['Nome'], row['ID'])

        for j, row2 in jcafb_cadastro.iterrows():

            if row2['Nome'] == row['Nome']:

                print(row2['Nome'], row2['ID'])

                new_row = {'Nome': row2['Nome'],
                           'ID': row2['ID'],
                           'Nascimento': row2['Nascimento'],
                           'Fonte': row2['Fonte'],
                          }
                homonimos.loc[len(homonimos)] = new_row

homonimos.to_excel(jcafb_homonimos_path, sheet_name=jcafb_homonimos_sheet, index=False)


Maria Aparecida de Lima 2
Maria Aparecida de Lima 073.172.458-52
Maria Aparecida de Lima 054.547.488-46
Maria Aparecida dos Santos 2
Maria Aparecida dos Santos 141.281.338-76
Maria Aparecida dos Santos 097.311.418-55


# Criação do arquivo para importação de "Street Patterns" pelo Odoo

## Criação do Arquivo

In [18]:
import pandas as pd

jcafb_cadastro_path = 'data/jcafb_cadastro.xlsx'
jcafb_cadastro_sheet = 'Cadastro JCAFB-2025'

street_pattern_path = 'data/street_pattern.xlsx'
street_pattern_sheet='Street Patterns'

column_names = ['id', 'street', 'street2', 'notes', 'active']
street_pattern = pd.DataFrame(columns=column_names)

xls_in = pd.ExcelFile(jcafb_cadastro_path)
jcafb_cadastro = pd.read_excel(xls_in, sheet_name=jcafb_cadastro_sheet, index_col=0, na_filter=False)

streets = jcafb_cadastro.groupby(['Street', 'Logradouro', 'Bairro'])['Nome'].aggregate('count').reset_index()

for i, row in streets.iterrows():

    if row['Street'] != '':

        _id = False
        street = False
        street2 = False
        notes = False
        active = 1

        pattern = row['Street'].lower().replace('(', '').replace(')', '').replace(' ', '_')

        _id = '__import__.clv_street_pattern_' + pattern
        street = row['Logradouro']
        street2 = row['Bairro']
        notes = row['Street']

        if not ((street2 == 'CH Manoel Rodrigues') or
                (street2 == 'Cohab')
               ):
            new_row = {'id': _id,
                       'street': street,
                       'street2': street2,
                       'notes': notes,
                       'active': active,
                      }
            street_pattern.loc[len(street_pattern)] = new_row

street_pattern.to_excel(street_pattern_path, sheet_name=street_pattern_sheet, index=False)

street_pattern

,id,street,street2,notes,active
0,__import__.clv_street_pattern_avenida_camilo_m...,Avenida Camilo Mady,Centro,Avenida Camilo Mady (Centro),1
1,__import__.clv_street_pattern_avenida_doutor_a...,Avenida Doutor Alberto Salles,Baixada,Avenida Doutor Alberto Salles (Baixada),1
2,__import__.clv_street_pattern_avenida_doutor_a...,Avenida Doutor Alberto Salles,Centro,Avenida Doutor Alberto Salles (Centro),1
3,__import__.clv_street_pattern_avenida_doutor_a...,Avenida Doutor Alberto Salles,NH Eunice Andrade,Avenida Doutor Alberto Salles (NH Eunice Andrade),1
4,__import__.clv_street_pattern_avenida_manoel_r...,Avenida Manoel Rodrigues,Centro,Avenida Manoel Rodrigues (Centro),1
...,...,...,...,...,...
56,__import__.clv_street_pattern_travessa_15_de_n...,Travessa 15 de Novembro,Vila Oliveira,Travessa 15 de Novembro (Vila Oliveira),1
57,__import__.clv_street_pattern_travessa_barao_d...,Travessa Barao do Rio Branco,Baixada,Travessa Barao do Rio Branco (Baixada),1
58,__import__.clv_street_pattern_travessa_leonild...,Travessa Leonilda Coelho de Azevedo,Baixada,Travessa Leonilda Coelho de Azevedo (Baixada),1
59,__import__.clv_street_pattern_travessa_manoel_...,Travessa Manoel Ramos,Vila Oliveira,Travessa Manoel Ramos (Vila Oliveira),1


# Criação do arquivo para importação de "Contact Information Patterns" pelo Odoo

## Criação do Arquivo

In [19]:
import pandas as pd

jcafb_cadastro_path = 'data/jcafb_cadastro.xlsx'
jcafb_cadastro_sheet = 'Cadastro JCAFB-2025'

contact_information_pattern_path = 'data/contact_information_pattern.xlsx'
contact_information_pattern_sheet='Contact Information Patterns'

column_names = ['id', 'name', 'street', 'street_number', 'street_number2', 'street2', 'notes', 'active']
contact_information_pattern = pd.DataFrame(columns=column_names)

xls_in = pd.ExcelFile(jcafb_cadastro_path)
jcafb_cadastro = pd.read_excel(xls_in, sheet_name=jcafb_cadastro_sheet, index_col=0, na_filter=False)

streets = jcafb_cadastro.groupby(['Contact', 'Logradouro', 'Número', 'Complemento', 'Bairro'])['Nome'].aggregate('count').reset_index()

for i, row in streets.iterrows():

    if (row['Logradouro'] != '') and (row['Número'].find('.') == -1) and (row['Complemento'].find("/") == -1):

        _id = False
        name = False
        street = False
        street_number = False
        street_number2 = False
        street2 = False
        notes = False
        active = 1

        pattern = row['Contact'].lower().replace('(', '').replace(')', '')
        pattern = pattern.replace(' ', '_').replace('.', '').replace(',', '')
        pattern = pattern.replace('/_', '_').replace('s/n', 'sn').replace('/', '_')

        _id = '__import__.clv_street_pattern_' + pattern
        name = row['Contact']
        street = row['Logradouro']
        street_number = row['Número']
        street_number2 = row['Complemento']
        street2 = row['Bairro']

        if not ((street2 == 'CH Manoel Rodrigues') or
                (street2 == 'Cohab')
               ):
            new_row = {'id': _id,
                       'name': name,
                       'street': street,
                       'street_number': street_number,
                       'street_number2': street_number2,
                       'street2': street2,
                       # 'notes': notes,
                       'active': active,
                      }
            contact_information_pattern.loc[len(contact_information_pattern)] = new_row

contact_information_pattern.to_excel(contact_information_pattern_path, sheet_name=street_pattern_sheet, index=False)

contact_information_pattern

,id,name,street,street_number,street_number2,street2,notes,active
0,__import__.clv_street_pattern_avenida_camilo_m...,"Avenida Camilo Mady, 102/Casa (Centro)",Avenida Camilo Mady,102,Casa,Centro,NaN,1
1,__import__.clv_street_pattern_avenida_camilo_m...,"Avenida Camilo Mady, 116/Casa (Centro)",Avenida Camilo Mady,116,Casa,Centro,NaN,1
2,__import__.clv_street_pattern_avenida_camilo_m...,"Avenida Camilo Mady, 15/Casa (Centro)",Avenida Camilo Mady,15,Casa,Centro,NaN,1
3,__import__.clv_street_pattern_avenida_camilo_m...,"Avenida Camilo Mady, 34/Casa (Centro)",Avenida Camilo Mady,34,Casa,Centro,NaN,1
4,__import__.clv_street_pattern_avenida_camilo_m...,"Avenida Camilo Mady, 83/Casa (Centro)",Avenida Camilo Mady,83,Casa,Centro,NaN,1
...,...,...,...,...,...,...,...,...
619,__import__.clv_street_pattern_travessa_manoel_...,"Travessa Manoel Ramos, 43/ (Vila Oliveira)",Travessa Manoel Ramos,43,,Vila Oliveira,NaN,1
620,__import__.clv_street_pattern_travessa_manoel_...,"Travessa Manoel Ramos, 68/ (Vila Oliveira)",Travessa Manoel Ramos,68,,Vila Oliveira,NaN,1
621,__import__.clv_street_pattern_viela_manoel_men...,"Viela Manoel Mendes, 31/Casa (Baixada)",Viela Manoel Mendes,31,Casa,Baixada,NaN,1
622,__import__.clv_street_pattern_viela_manoel_men...,"Viela Manoel Mendes, 40/Casa (Baixada)",Viela Manoel Mendes,40,Casa,Baixada,NaN,1


# Criação do arquivo para importação do Cadastro pelo Odoo

## Funções de Suporte (3)

In [20]:
import re

def somaPoderada(numero):
    i = 0
    soma = 0
    while i<len(numero):
        soma = soma+int(numero[i]) * (15 - i)
        i = i+1
    return soma

def validaCNS(numero):
    numero = str(numero)
    if numero.isdigit():
        if re.match(r'[1-2]\d{10}00[0-1]\d$',numero) or re.match(r'[7-9]\d{14}$',numero):
            return somaPoderada(numero) % 11 == 0
    return False

validaCNS('703008820145471')
# validaCNS('702803658947067')
# validaCNS('704301554368493')
# validaCNS('700506784991850')


True

In [21]:
def format_code(code_seq):
    code = list(map(int, str(code_seq)))
    code_len = len(code)
    while len(code) < 14:
        code.insert(0, 0)
    while len(code) < 16:
        n = sum([(len(code) + 1 - i) * v for i, v in enumerate(code)]) % 11
        if n > 1:
            f = 11 - n
        else:
            f = 0
        code.append(f)
    code_str = "%s.%s.%s.%s.%s-%s" % (str(code[0]) + str(code[1]),
                                      str(code[2]) + str(code[3]) + str(code[4]),
                                      str(code[5]) + str(code[6]) + str(code[7]),
                                      str(code[8]) + str(code[9]) + str(code[10]),
                                      str(code[11]) + str(code[12]) + str(code[13]),
                                      str(code[14]) + str(code[15]))
    if code_len <= 3:
        code_form = code_str[18 - code_len:21]
    elif code_len > 3 and code_len <= 6:
        code_form = code_str[17 - code_len:21]
    elif code_len > 6 and code_len <= 9:
        code_form = code_str[16 - code_len:21]
    elif code_len > 9 and code_len <= 12:
        code_form = code_str[15 - code_len:21]
    elif code_len > 12 and code_len <= 14:
        code_form = code_str[14 - code_len:21]
    return code_form

def validaCPF(code):

    if code is not False:

        sequence_str = ''
        for c in code:
            if c.isdigit():
                sequence_str = sequence_str + c
        sequence_str = sequence_str[:len(sequence_str) - 2]
        _format_code = format_code(sequence_str)

        if code == _format_code:
            return True

    return False

validaCPF('551.583.048-23')
# validaCPF('525.338.988-13')


True

## Validação dos IDs

In [22]:
import pandas as pd

jcafb_cadastro_path = 'data/jcafb_cadastro.xlsx'
jcafb_cadastro_sheet='Cadastro JCAFB-2025'

xls_in = pd.ExcelFile(jcafb_cadastro_path)
jcafb_cadastro = pd.read_excel(xls_in, sheet_name=jcafb_cadastro_sheet, index_col=0, na_filter=False)

column_names = ['Nome', 'ID', 'Fonte']
invalid_IDs = pd.DataFrame(columns=column_names)

for i, row in jcafb_cadastro.iterrows():

    _id = row['ID']
    validate = False
    # print(_id, validaCNS(_id), validaCPF(_id))
    if validaCNS(_id):
        validate = True
    if not validate:
        if validaCPF(_id):
            validate = True
    if not validate:
        new_row = {'Nome': row['Nome'],
                   'ID': _id,
                   'Fonte': row['Fonte']
                  }
        invalid_IDs.loc[len(invalid_IDs)] = new_row

invalid_IDs


,Nome,ID,Fonte


## Criação do Arquivo

In [23]:
import pandas as pd

patient_marker_path = 'data/patient_marker.xlsx'
patient_marker_sheet='Sheet1'

column_names = ['id', 'name', 'description']

markers = {
    'id': ['__import__.clv_patient_marker_MC1_Criancas',
           '__import__.clv_patient_marker_MC1_Idosos',
           '__import__.clv_patient_marker_MC2_Criancas',
           '__import__.clv_patient_marker_MC2_Idosos',
           '__import__.clv_patient_marker_MC3_Criancas',
           '__import__.clv_patient_marker_MC3_Idosos',
           '__import__.clv_patient_marker_MC4_Criancas',
           '__import__.clv_patient_marker_MC4_Idosos',
           '__import__.clv_patient_marker_MC5_Criancas',
           '__import__.clv_patient_marker_MC5_Idosos',
           '__import__.clv_patient_marker_MC6_Criancas',
           '__import__.clv_patient_marker_MC6_Idosos',
           '__import__.clv_patient_marker_MC7_Criancas',
           '__import__.clv_patient_marker_MC7_Idosos',
    ],
    'name': ['MC1_Criancas', 'MC1_Idosos',
             'MC2_Criancas', 'MC2_Idosos',
             'MC3_Criancas', 'MC3_Idosos',
             'MC4_Criancas', 'MC4_Idosos',
             'MC5_Criancas', 'MC5_Idosos',
             'MC6_Criancas', 'MC6_Idosos',
             'MC7_Criancas', 'MC7_Idosos',
    ],
    'description': ['Micro Área 1 - Crianças',
                    'Micro Área 1 - Idosos',
                    'Micro Área 2 - Crianças',
                    'Micro Área 2 - Idosos',
                    'Micro Área 3 - Crianças',
                    'Micro Área 3 - Idosos',
                    'Micro Área 4 - Crianças',
                    'Micro Área 4 - Idosos',
                    'Micro Área 5 - Crianças',
                    'Micro Área 5 - Idosos',
                    'Micro Área 6 - Crianças',
                    'Micro Área 6 - Idosos',
                    'Micro Área 7 - Crianças',
                    'Micro Área 7 - Idosos',
    ]
}
patient_marker = pd.DataFrame(markers, columns=column_names)

patient_marker.to_excel(patient_marker_path, sheet_name=patient_marker_sheet, index=False)


In [24]:
jcafb_cadastro_path = 'data/jcafb_cadastro.xlsx'
jcafb_cadastro_sheet = 'Cadastro JCAFB-2025'

jcafb_cadastro_odoo_path = 'data/jcafb_cadastro_odoo.xlsx'
jcafb_cadastro_odoo_sheet = 'Sheet1'

column_names = ['id', 'name', 'code', 'gender', 'birthday', 'street', 'street_number',
                'street_number2', 'street2', 'zip', 'city_id/id', 'state_id/id', 'country_id/id',
                'phone', 'mobile', 'email', 'marker_ids/id', 'phase_id']
cadastr_odoo = pd.DataFrame(columns=column_names)

xls_in = pd.ExcelFile(jcafb_cadastro_path)
jcafb_cadastro = pd.read_excel(xls_in, sheet_name=jcafb_cadastro_sheet, index_col=0, na_filter=False)

for i, row in jcafb_cadastro.iterrows():

    _id = False
    name = False
    _code = row['ID']
    code = False
    gender = False
    birthday = False
    street = False
    street_number = False
    street_number2 = False
    street_number2 = False
    _zip = False
    city = False
    state = False
    country = False
    phone = False
    marker_ids = False
    phase_id = 'JCAFB-2025'
    # state = 'Available'

    sequence_str = ''
    for c in _code:
        if c.isdigit():
            sequence_str = sequence_str + c

    _id = '__import__.clv_patient_' + sequence_str
    name = row['Nome']
    code = _code
    if row['Sexo'] == 'Masculino':
        gender = 'Male'
    if row['Sexo'] == 'Feminino':
        gender = 'Female'
    birthday = row['Nascimento']
    street = row['Logradouro']
    street_number = row['Número']
    street_number2 = row['Complemento']
    street2 = row['Bairro']
    _zip = '16680-000'
    city = 'clv_l10n_br_base.city_3504305'
    state_id = 'base.state_br_sp'
    country = 'base.br'
    phone = row['Telefone']
    if row['Fonte'] == 'MC1_Criancas':
        marker_ids = '__import__.clv_patient_marker_MC1_Criancas'
    if row['Fonte'] == 'MC1_Idosos':
        marker_ids = '__import__.clv_patient_marker_MC1_Idosos'
    if row['Fonte'] == 'MC2_Criancas':
        marker_ids = '__import__.clv_patient_marker_MC2_Criancas'
    if row['Fonte'] == 'MC2_Idosos':
        marker_ids = '__import__.clv_patient_marker_MC2_Idosos'
    if row['Fonte'] == 'MC3_Criancas':
        marker_ids = '__import__.clv_patient_marker_MC3_Criancas'
    if row['Fonte'] == 'MC3_Idosos':
        marker_ids = '__import__.clv_patient_marker_MC3_Idosos'
    if row['Fonte'] == 'MC4_Criancas':
        marker_ids = '__import__.clv_patient_marker_MC4_Criancas'
    if row['Fonte'] == 'MC4_Idosos':
        marker_ids = '__import__.clv_patient_marker_MC4_Idosos'
    if row['Fonte'] == 'MC5_Criancas':
        marker_ids = '__import__.clv_patient_marker_MC5_Criancas'
    if row['Fonte'] == 'MC5_Idosos':
        marker_ids = '__import__.clv_patient_marker_MC5_Idosos'
    if row['Fonte'] == 'MC6_Criancas':
        marker_ids = '__import__.clv_patient_marker_MC6_Criancas'
    if row['Fonte'] == 'MC6_Idosos':
        marker_ids = '__import__.clv_patient_marker_MC6_Idosos'
    if row['Fonte'] == 'MC7_Criancas':
        marker_ids = '__import__.clv_patient_marker_MC7_Criancas'
    if row['Fonte'] == 'MC7_Idosos':
        marker_ids = '__import__.clv_patient_marker_MC7_Idosos'

    new_row = {'id': _id,
               'name': name,
               'code': code,
               'gender': gender,
               'birthday': birthday,
               'street': street,
               'street_number': street_number,
               'street_number2': street_number2,
               'street2': street2,
               'zip': _zip,
               'city_id/id': city,
               'state_id/id': state_id,
               'country_id/id': country,
               'phone': phone,
               'marker_ids/id': marker_ids,
               'phase_id': phase_id,
               # 'state': state,
              }
    cadastr_odoo.loc[len(cadastr_odoo)] = new_row

cadastr_odoo.to_excel(jcafb_cadastro_odoo_path, sheet_name=jcafb_cadastro_odoo_sheet, index=False)
